# An Introduction to WISER: Part 1 - Labeling Rules

Welcome to WISER (*Weak and Indirect Supervision for Entity Recognition*), a system for training sequence-to-sequence tagging models, particularly neural networks for named entity recognition (NER) and related tasks.

WISER uses *weak supervision* in the form of rules to train these models, as opposed to hand-labeled training data.

## Loading Data
WISER is an add-on to [AllenNLP](http://allennlp.org), a great framework for natural language processing. That means we can use their tools for working with data.

__TODO: pointers to prerequisite installation__

Let's start by loading the [CoNLL 2003](https://www.clips.uantwerpen.be/conll2003/ner/) dataset, a common benchmark for NER.

In [1]:
import sys
sys.path.append('../..')

In [2]:
%%capture
from allennlp.data.dataset_readers.conll2003 import Conll2003DatasetReader

dataset_reader = Conll2003DatasetReader(coding_scheme='IOB1')
training_data = dataset_reader.read('../data/conll/eng.train')
dev_data = dataset_reader.read('../data/conll/eng.testa')

# We must merge all data partitions to simultaneously apply weak supervision rules to all of them
data = training_data + dev_data

The easiest way to use WISER with other data sets is to implement a new subclass of AllenNLP's [DatasetReader](https://allenai.github.io/allennlp-docs/api/allennlp.data.dataset_readers.dataset_reader.html#allennlp.data.dataset_readers.dataset_reader.DatasetReader). We have some additional examples in the package `wiser.data.dataset_readers`.

## Inspecting Data
Once the data is loaded, we use a WISER class called `Viewer` to inspect the sentences and tags.

In [3]:
from wiser.viewer import Viewer
Viewer(data, height=100)

<IPython.core.display.Javascript object>

Viewer(html='<head>\n<style>\nspan.active {\n    background-color: skyblue;\n    box-shadow: 1px 1px 1px grey;…

You can use the left and right buttons to flip through the items in `dev_data`, each of which is an AllenNLP [`Instance`](https://allenai.github.io/allennlp-docs/api/allennlp.data.instance.html#allennlp.data.instance.Instance). The highlighted spans are the entities, and you can hover over each one with your cursor to see whether it is a person (PER), location (LOC), organization (ORG), or miscellaneous (MISC).

The drop-down menu selects which source of labels is displayed. Currently only the gold labels from the benchmark are available, but we will add more soon.

Advance to the instance at index 2 to see an example with multiple entities of different classes. You can access the underlying tokens and tags too.

Notice that WISER uses the [IOB1 tagging scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), meaning that entities are represented as consecutive tags beginning with `I`. Many data sets use subsequent characters for different classes, for example `-MISC` and `-PER` here for miscellaneous and person, respectively. The `O` tag means that the token is not part of an entity. There is also a special set of tags beginning with `B` (like those beginning with `I`) that are used to start a new entity that immediately follows another of the same class without an `O` tag in between.

## Writing Labeling Functions

Inspecting the data will give you an idea of the tagging structure of sentences. Therefore, we can start writing our own labeling functions. 

In [4]:
from wiser.lf import LabelingFunction

In [5]:
class ConsecutiveCapitalization(LabelingFunction):
    
    def apply_instance(self, instance):
        labels = ['ABS'] * len(instance['tokens'])
        
        for i in range(1, len(instance['tokens'])):
            if instance['tokens'][i].text.istitle():
                labels[i] = "I-PER"
        return labels

lf = ConsecutiveCapitalization()
lf.apply(data)

You can also use existing labeling function helpers available at `wiser.lf`

In [6]:
from wiser.lf import DictionaryMatcher

In [7]:
# Tags all countries found in an external dataset as 'I-LOC'
import csv
with open('../data/countries.csv') as csvfile:
    country_list = list(csv.reader(csvfile))

lf = DictionaryMatcher("Countries", terms=country_list, uncased=True, i_label="I-LOC")
lf.apply(data)

In [8]:
# Tags all punctuations signs as 'O'
punctuation_chars = {'.', ',', ':', ';', '-', '?', '!', '@', '$'}

lf = DictionaryMatcher("Punctuation", terms=punctuation_chars, i_label="O")
lf.apply(data)

# Evaluating Labeling Functions

We can evalualte labeling functions on the development set in either of two ways. First, we can inspect individual labeling functions using the ``score_individual_labeling_functions`` method.

In [9]:
from wiser.lf import score_individual_labeling_functions
score_individual_labeling_functions(dev_data)

,TP,FP,FN,Token Acc.,Token Votes
ConsecutiveCapitalization,1348,4110,4595,0.3625,7498
Countries,712,46,5231,0.9538,758
Punctuation,0,0,5943,0.9993,4463


We can also look inspect at the precision, recall, and F1 scores of the combined labeling rules with ``score_labeling_functions_majority_vote``

In [10]:
from wiser.lf import score_labeling_functions_majority_vote
score_labeling_functions_majority_vote(dev_data)

,TP,FP,FN,P,R,F1
Majority Vote,1494,3552,4449,0.2961,0.2514,0.2719


# Saving Progress
We can use pickle to store the data with the labeling function applied to it

In [11]:
import pickle

with open('tmp/training_data.p', 'wb') as f:
    pickle.dump(training_data, f)

with open('tmp/dev_data.p', 'wb') as f:
    pickle.dump(dev_data, f)

You have completed part 1! Now you can move on to part 2.